## Test Script for Remote Simulations

In [56]:
# Standard Package imports
import numpy as np
import numpy.linalg as la
import numpy.random as rm
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sys
import os

In [ ]:
'Instantons/Rotated-2D-Well/Stochastic-Model/Data/eps0_01/alpha_0_0/cold-ensemble'

In [122]:
##########################################
## Setting Parameters
##########################################

alphas = [0., 0.25, 0.5, 1.]
alpha = alphas[int(sys.argv[1]) - 1] # For use with array jobs

eps = 0.01

dt = 0.1
tf = 10
time = np.arange(0, tf, dt)

p = [alpha, eps]

In [124]:
##########################################
## Save Details
##########################################

# Results saved after each block
# Ensemble size = blocks * block_len
# Half of the ensemble start at the cold point, half at the hot point
blocks = 10
block_len = 10

# Where We Save Output
save_directory = f'/rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_{alpha}/'.replace('.', '_')

if not os.path.exists(save_directory):
    os.makedirs(save_directory + '/cold-ensemble')
    os.makedirs(save_directory + '/hot-ensemble')

In [125]:
##########################################
## E.M. Scheme Definition
##########################################

R = np.array([[0, -1], [1, 0]]) # 90 degree rotation matrix

def grad_V(x):
    return np.array([x[0]*(x[0]**2 -1), 2 * x[1]])

def drift(x, p):
    alpha, eps = p
    return - (np.eye(2) + alpha * R) @ grad_V(x)

def euler_maruyama(x0, t, p, timer=False):
    alpha, eps = p
    N = len(t)
    x = np.zeros(np.append(N, x0.shape))
    x[0] = x0
    for i in tqdm(range(N-1), disable=not timer):
        dt = t[i+1]-t[i]
        dWt = rm.normal(0, np.sqrt(dt), 2)
        x[i+1] = x[i] + drift(x[i], p) * dt + np.sqrt(eps) * dWt
    return x

In [126]:
cold_point = np.array([-1, 0])
hot_point = np.array([1, 0])
saddle = np.array([0, 0])

## Running Ensemble of Stochastic Integrations

In [127]:
##########################################
## Running and Saving in Blocks
##########################################

cold_ensemble = []
hot_ensemble = []

bl = int(block_len/2) # half of points start in each basin

for i in range(blocks):
    
    for k in range(bl):
        ensemble_member = bl * i + k + 1
        cold_ensemble.append(euler_maruyama(cold_point, time, p))
        hot_ensemble.append(euler_maruyama(hot_point, time, p))
        
    # Save and clear
    
    save_ensemble(cold_ensemble, save_directory + f'cold-ensemble/{i + 1}')
    save_ensemble(cold_ensemble, save_directory + f'hot-ensemble/{i + 1}')
    cold_ensemble = []
    hot_ensemble = []

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/cold-ensemble/1

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/hot-ensemble/1

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/cold-ensemble/2

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/hot-ensemble/2

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/cold-ensemble/3

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/hot-ensemble/3

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/cold-ensemble/4

Saved at /rds/general/user/cfn18/home/Instantons/Rotated-2D-Well/Stochastic-Model/Data/Test/alpha_1_0/hot-ensemble/4

Saved at /rds/general/user/cfn18/home/Instantons/Rot

In [ ]:
# Putting Results in xarray dataset and Saving

def save_ensemble(ensemble, save_name):

    # xr setup
    dims = ['x', 'y']
    coords = [time]
    attrs = {'alpha': alpha, 'eps': eps}
    
    # Creating xr.Dataset
    data = np.asarray(ensemble)
    x_data = xr.DataArray(data[:, :, 0], coords = {'realisation':np.arange(len(ensemble)) + 1,
                                        'time': time},
                        dims = ['realisation','time'], attrs=attrs, name='x')
    y_data = xr.DataArray(data[:, :, 1], coords = {'realisation':np.arange(len(ensemble)) + 1,
                                        'time': time},
                        dims = ['realisation','time'], attrs=attrs, name='y')
    ensemble_ds = xr.merge([x_data, y_data])
    ensemble_ds.attrs = attrs
    
    # Saving Dataset
    ensemble_ds.to_netcdf(save_name + '.nc')
    print(f'Saved at {save_name}\n')